In [11]:
import nannyml as nml
from IPython.display import display

reference_df = nml.load_synthetic_car_loan_dataset()[0]
analysis_df = nml.load_synthetic_car_loan_dataset()[1]

display(reference_df.head(3))

,car_value,salary_range,debt_to_income_ratio,loan_length,repaid_loan_on_prev_car,size_of_downpayment,driver_tenure,repaid,timestamp,y_pred_proba,y_pred
0,39811.0,40K - 60K €,0.632950,19.0,False,40%,0.212653,1.0,2018-01-01 00:00:00.000,0.99,1
1,12679.0,40K - 60K €,0.718627,7.0,True,10%,4.927549,0.0,2018-01-01 00:08:43.152,0.07,0
2,19847.0,40K - 60K €,0.721724,17.0,False,0%,0.520817,1.0,2018-01-01 00:17:26.304,1.00,1


In [12]:
print(reference_df.head(3).to_markdown(tablefmt="grid"))

+----+-------------+----------------+------------------------+---------------+---------------------------+-----------------------+-----------------+----------+-------------------------+----------------+----------+
|    |   car_value | salary_range   |   debt_to_income_ratio |   loan_length | repaid_loan_on_prev_car   | size_of_downpayment   |   driver_tenure |   repaid | timestamp               |   y_pred_proba |   y_pred |
+====+=============+================+========================+===============+===========================+=======================+=================+==========+=========================+================+==========+
|  0 |       39811 | 40K - 60K €    |               0.63295  |            19 | False                     | 40%                   |        0.212653 |        1 | 2018-01-01 00:00:00.000 |           0.99 |        1 |
+----+-------------+----------------+------------------------+---------------+---------------------------+-----------------------+--------------

In [13]:
estimator = nml.CBPE(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='repaid',
    timestamp_column_name='timestamp',
    metrics=['roc_auc', 'confusion_matrix', 'business_value'],
    chunk_size=5000,
    problem_type='classification_binary',
    normalize_confusion_matrix='pred',
    business_value_matrix=[[2, -5], [-10, 10]],
    normalize_business_value='per_prediction'
)

estimator.fit(reference_df)

In [14]:
results = estimator.estimate(analysis_df)
display(results.filter(period='analysis').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-10-30 18:00:00   
1    [5000:9999]           1        5000      9999 2018-11-30 00:36:00   
2  [10000:14999]           2       10000     14999 2018-12-30 07:12:00   
3  [15000:19999]           3       15000     19999 2019-01-29 13:48:00   
4  [20000:24999]           4       20000     24999 2019-02-28 20:24:00   
5  [25000:29999]           5       25000     29999 2019-03-31 03:00:00   
6  [30000:34999]           6       30000     34999 2019-04-30 09:36:00   
7  [35000:39999]           7       35000     39999 2019-05-30 16:12:00   
8  [40000:44999]           8       40000     44999 2019-06-29 22:48:00   
9  [45000:49999]           9       45000     49999 2019-07-30 05:24:00   

                                      roc_auc                          ...  \
                 end_date    period     value sampling_error realized  ...   
0 2018-11-30 00:27:16.848  analysis  0.968631       0.001811      NaN  ...   
1 2018-12-30 07:03:16.848  analysis  0.969044       0.001811      NaN  ...   
2 2019-01-29 13:39:16.848  analysis  0.969444       0.001811      NaN  ...   
3 2019-02-28 20:15:16.848  analysis  0.969047       0.001811      NaN  ...   
4 2019-03-31 02:51:16.848  analysis  0.968873       0.001811      NaN  ...   
5 2019-04-30 09:27:16.848  analysis  0.960478       0.001811      NaN  ...   
6 2019-05-30 16:03:16.848  analysis  0.961134       0.001811      NaN  ...   
7 2019-06-29 22:39:16.848  analysis  0.960536       0.001811      NaN  ...   
8 2019-07-30 05:15:16.848  analysis  0.961869       0.001811      NaN  ...   
9 2019-08-29 11:51:16.848  analysis  0.960537       0.001811      NaN  ...   

   false_negative        business_value                          \
  lower_threshold  alert          value sampling_error realized   
0        0.056719  False       5.261802       0.072062      NaN   
1        0.056719  False       5.056792       0.072062      NaN   
2        0.056719  False       5.091166       0.072062      NaN   
3        0.056719  False       5.103276       0.072062      NaN   
4        0.056719  False       5.184856       0.072062      NaN   
5        0.056719   True       4.762307       0.072062      NaN   
6        0.056719   True       4.747993       0.072062      NaN   
7        0.056719   True       4.748050       0.072062      NaN   
8        0.056719   True       4.737356       0.072062      NaN   
9        0.056719   True       4.877510       0.072062      NaN   

                                                                       \
  upper_confidence_boundary lower_confidence_boundary upper_threshold   
0                  5.477989                  5.045614        5.298755   
1                  5.272979                  4.840604        5.298755   
2                  5.307354                  4.874979        5.298755   
3                  5.319464                  4.887089        5.298755   
4                  5.401044                  4.968669        5.298755   
5                  4.978494                  4.546119        5.298755   
6                  4.964181                  4.531806        5.298755   
7                  4.964238                  4.531863        5.298755   
8                  4.953543                  4.521168        5.298755   
9                  5.093698                  4.661323        5.298755   

                          
  lower_threshold  alert  
0        4.984045  False  
1        4.984045  False  
2        4.984045  False  
3        4.984045  False  
4        4.984045  False  
5        4.984045   True  
6        4.984045   True  
7        4.984045   True  
8        4.984045   True  
9        4.984045   True  

[10 rows x 55 columns]

In [15]:
from docs.utils import print_multi_index_markdown
print_multi_index_markdown(results.filter(period='analysis').to_df())

+----+---------------+-----------------+-----------------+---------------+---------------------+----------------------------+------------+-------------+--------------------+--------------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+-------------------+--------------------+--------------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+-------------------+--------------------+--------------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+--------------------+--------------------+--------------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+--------------------+--------------------+--------------+-------------------------------+-------------------------------+---------------------+--------------------

In [16]:
metric_fig = results.filter(period='analysis').plot()
metric_fig.show()

In [20]:
metric_fig = results.filter(period='analysis').plot()
metric_fig.write_image(file=f"../_static/tutorials/performance_estimation/binary/tutorial-performance-estimation-binary-car-loan-analysis.svg")

In [18]:
metric_fig = results.plot()
metric_fig.show()

In [19]:
metric_fig = results.plot()
metric_fig.write_image(file=f"../_static/tutorials/performance_estimation/binary/tutorial-performance-estimation-binary-car-loan-analysis-with-ref.svg")
